In [1]:
import os
import pandas as pd
from tqdm import tqdm

In [2]:
# ==== USER CONFIG ====
zero_corr_file = r"C:\Users\USER\Desktop\Jupyter Notebook\Stelco\zero correlation.xlsx"
master_folder = r"D:\Stelco\Work\Dynamic Correlation\Key\Master whole Phase"
output_file = "reverse_search_corr_above_15_batch.csv"

batch_size = 300   # How many (signal, shape) pairs per run
start_idx = 200      # Start from this index in the unique pairs list

In [3]:
#Load zero correlation pairs
zero_df = pd.read_excel(zero_corr_file, sheet_name='Sheet1')

# Unique pairs
pairs_all = zero_df[['Signal Feature', 'Shape Feature']].drop_duplicates().values.tolist()

# Batch slice
pairs_batch = pairs_all[start_idx : start_idx + batch_size]

print(f"Processing batch: pairs {start_idx} to {start_idx + len(pairs_batch)-1} "
      f"out of {len(pairs_all)} total pairs.")

result_list = []

# oil files list
coil_files = [f for f in os.listdir(master_folder) if f.endswith(".csv")]

# Process each coil
for coil_file in tqdm(coil_files, desc="Processing coil files", unit="coil"):
    coil_id = os.path.splitext(coil_file)[0]
    file_path = os.path.join(master_folder, coil_file)

    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"Error reading {coil_file}: {e}")
        continue

    # Check each pair in the batch
    for signal_feature, shape_feature in pairs_batch:
        col_name = f"{signal_feature}_Rolling_Corr_{shape_feature}"
        if col_name in df.columns:
            vals_above_15 = df.loc[df[col_name] > 15, col_name]
            if not vals_above_15.empty:
                for val in vals_above_15:
                    if pd.notnull(val):
                        result_list.append({
                            "Coil ID": coil_id,
                            "Signal Feature": signal_feature,
                            "Shape Feature": shape_feature,
                            "Correlation Value": val
                        })

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\USER\\Desktop\\Jupyter Notebook\\Stelco\\zero correlation.xlsx'

In [ ]:
# Save this batch
result_df = pd.DataFrame(result_list)
result_df.to_csv(output_file, index=False, mode='a', header=not os.path.exists(output_file))

print(f"Finished batch {start_idx}–{start_idx + len(pairs_batch)-1}. "
      f"Output saved to: {output_file}")

Processing batch: pairs 200 to 484 out of 485 total pairs.


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_4332\2101165659.py:37: DtypeWarning: Columns (8,9,12,14,33,35,42,46,47,49,50,51,53,55,66,70,75,79,80,84,85,87,93,104,116,125,136,139,142) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_4332\2101165659.py:37: DtypeWarning: Columns (8,9,10,12,14,28,29,31,33,35,42,46,47,49,50,51,53,55,61,66,67,70,74,75,79,80,83,84,85,87,93,96,104,106,116,118,119,123,124,125,126,127,129,135,136,139,142) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
Processing coil files: 100%|█████████████████████████████████████████████████████| 2988/2988 [42:07<00:00,  1.18coil/s]

Finished batch 200–484. Output saved to: reverse_search_corr_above_15_batch.csv
